In [1]:
import numpy as np
import os
from biopandas.pdb import PandasPdb
import pandas as pd
import random
import itertools
import copy
import scipy
import math
import sys
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.transform import Rotation as R
from scipy.spatial.distance import directed_hausdorff, pdist, squareform, cdist
from scipy.spatial import KDTree
import matplotlib.pyplot as plt

from scipy.spatial.transform import Rotation as R
from scipy.spatial.distance import directed_hausdorff, cdist
from scipy.optimize import minimize
from itertools import product
from joblib import Parallel, delayed
import multiprocessing

In [2]:
def min_haus(degs, L, D):
    
    x,y,z,a,b,c = degs   
    
    r = R.from_euler('xyz', [a,b,c])
    D = r.apply(D)
    
    D += [x,y,z]
    haus, _, _ = directed_hausdorff(L, D)
    
    return haus

def haus_opt(x0, L, D):

    res = minimize(min_haus, x0, method='SLSQP', args=(L,D), options={'ftol': 1e-3, 'disp': False})

    return res['fun'], res['x']

def hcm(L):
    D = -L
    perm = product(np.linspace(-np.pi, np.pi, 10), repeat=3) #from -pi to pi rads, 24 even groups in between them 
    inits_abc = np.array([i for i in perm])
    # iter_xyz = product(np.linspace(-np.max(cdist(c1_xyz, c1_xyz)) /50, np.max(cdist(c1_xyz, c1_xyz)) /50, 3), repeat=3)
    # inits_xyz = np.array([i for i in iter_xyz])
    inits_xyz = np.zeros([1,3])
    inits = np.append(np.tile(inits_xyz, [len(inits_abc), 1]), np.repeat(inits_abc, len(inits_xyz), axis=0), axis=1)

    num_cores = multiprocessing.cpu_count()
    output = Parallel(n_jobs=num_cores)(delayed(haus_opt)(i, L, D) for i in inits)
    values = [_[0] for _ in output]
    return (min(values) / np.max(cdist(L,L)))

In [3]:
def CA_coord(pdb_name, chain1, chain2):

    coord1 = PandasPdb()
    coord1.fetch_pdb(pdb_name)
    # coord1.read_pdb(os.path.join(protein_dir, pdb_name + '.pdb'))
    prot1_df = coord1.df['ATOM']
    prot1_df = prot1_df[(prot1_df['alt_loc'] == "") | (prot1_df['alt_loc'] == "A")]
    
    c1_ = [[] for _ in range(len(chain1))]
    for ii in range(len(chain1)):
        c1_[ii] = prot1_df[prot1_df['chain_id'] == chain1[ii]]
    c1 = pd.concat(c1_).reset_index(drop=True)
    
    c1_all_res = c1[['chain_id', 'residue_number', 'insertion']].drop_duplicates().reset_index(drop=True)
    c1_ca_res = c1[c1['atom_name'] == 'CA'][['chain_id', 'residue_number', 'insertion']]
    c1_no_cas = pd.merge(c1_all_res, c1_ca_res, how='left', indicator=True)['_merge']=='left_only'
    
    if sum(c1_no_cas) != 0:
        c1_incomplete_res = np.squeeze(np.where(c1[['chain_id', 'residue_number', 'insertion']].astype(str).agg('_'.join, axis=1).to_numpy() ==
               				        c1_all_res[c1_no_cas].astype(str).agg('_'.join, axis=1).to_numpy()))
        c1 = c1.drop(np.atleast_1d((c1_incomplete_res))).reset_index(drop=True)
    else:
        c1_incomplete_res = np.array([])
    
    c2_ = [[] for _ in range(len(chain2))]
    for ii in range(len(chain2)):
        c2_[ii] = prot1_df[prot1_df['chain_id'] == chain2[ii]]
    c2 = pd.concat(c2_).reset_index(drop=True)

    c2_all_res = c2[['chain_id', 'residue_number', 'insertion']].drop_duplicates().reset_index(drop=True)
    c2_ca_res = c2[c2['atom_name'] == 'CA'][['chain_id', 'residue_number', 'insertion']]
    c2_no_cas = pd.merge(c2_all_res, c2_ca_res, how='left', indicator=True)['_merge'] == 'left_only'

    if sum(c2_no_cas) != 0:
        c2_incomplete_res = np.squeeze(np.where(c2[['chain_id', 'residue_number', 'insertion']].astype(str).agg('_'.join, axis=1).to_numpy() ==
                     c2_all_res[c2_no_cas].astype(str).agg('_'.join, axis=1).to_numpy()))
        c2 = c2.drop(np.atleast_1d((c2_incomplete_res))).reset_index(drop=True)
    else:
        c2_incomplete_res = np.array([])
    
    c1_CA = pd.concat([c1[c1['atom_name'] == 'CA']['x_coord'],
                       c1[c1['atom_name'] == 'CA']['y_coord'],
                       c1[c1['atom_name'] == 'CA']['z_coord']],
                      axis=1).to_numpy()
    c2_CA = pd.concat([c2[c2['atom_name'] == 'CA']['x_coord'],
                       c2[c2['atom_name'] == 'CA']['y_coord'],
                       c2[c2['atom_name'] == 'CA']['z_coord']],
                      axis=1).to_numpy()

    return c1, c2, c1_CA, c2_CA, c1_incomplete_res, c2_incomplete_res

In [4]:
def get_radius(A, p, r):
    dists = cdist(A,p.reshape(1,3)).reshape(-1)
    within_cutoff = (dists <= r)
    selection = A[within_cutoff]
    return selection

In [5]:
def osipov_whole(coord):
    n = 2
    m = 1
    
    N = len(coord)
    
    if N >= 4:
        P = np.array(list(itertools.permutations(np.arange(N), 4))) # Get permutations
        
        coords_P = coord[P]
        r = coords_P - np.roll(coords_P, -1, axis=1)
        r[:, 3] = -r[:, 3]
        r_mag = np.linalg.norm(r, axis=-1)
    
        cross_vecs = np.cross(r[:, 0], r[:, 2])
    
        G_p_up = np.einsum('ij,ij->i', cross_vecs, r[:, 3]) * np.einsum('ij,ij->i', r[:, 0], r[:, 1]) * np.einsum('ij,ij->i', r[:, 1], r[:, 2])
        G_p_down = np.power(np.prod(r_mag[:,0:3], axis=-1), n) * np.power(r_mag[:, 3], m)
        
        
        G_p = (1 / 3) * np.sum(G_p_up / G_p_down)
    
        G_os = (24)/(N ** 4) *  G_p
        
    else:
        G_os = 0

    return G_os

In [6]:
def get_chirality_scale_matrix_hcm(c1):
    c2 = c1[c1['atom_name'] == 'CA']
    
    # All non-hydrogen atoms in protein
    chain1 = c1
    chain1_arr = np.asarray(chain1[['atom_number', 'x_coord', 'y_coord', 'z_coord']])
    A1 = np.asarray(chain1[['x_coord', 'y_coord', 'z_coord']])

    # All carbon alphas in protein
    chain2 = c2
    chain2_arr = np.asarray(chain2[['atom_number', 'x_coord', 'y_coord', 'z_coord']])
    A2 = np.asarray(chain2[['x_coord', 'y_coord', 'z_coord']])

    # Compute diamater of protein
    distances = squareform(pdist(A1))
    diameter = distances.max()

    radii = np.array([2,4,6,8,10])
    vector_list = []

    for row in A2:
        row_vec = np.asarray([hcm(get_radius(A1, row, r)) for r in radii])
        vector_list.append(row_vec)

    result_arr = np.vstack(vector_list)
    result_df = pd.DataFrame(result_arr)
    result_df.columns = radii
    result_df.index = chain2_arr[:, 0]

    return result_df

In [7]:
def get_chirality_scale_matrix_opd(c1):
    c2 = c1[c1['atom_name'] == 'CA']
    
    # All non-hydrogen atoms in protein
    chain1 = c1[c1['element_symbol'] != 'H']
    chain1_arr = np.asarray(chain1[['atom_number', 'x_coord', 'y_coord', 'z_coord']])
    A1 = np.asarray(chain1[['x_coord', 'y_coord', 'z_coord']])

    # All carbon alphas in protein
    chain2 = c2
    chain2_arr = np.asarray(chain2[['atom_number', 'x_coord', 'y_coord', 'z_coord']])
    A2 = np.asarray(chain2[['x_coord', 'y_coord', 'z_coord']])

    # Compute diamater of protein
    distances = squareform(pdist(A1))
    diameter = distances.max()

    radii = np.array([3])
    vector_list = []

    for row in A2:
        row_vec = np.asarray([osipov_whole(get_radius(A1, row, r)) for r in radii])
        vector_list.append(row_vec)

    result_arr = np.vstack(vector_list)
    result_df = pd.DataFrame(result_arr)
    result_df.columns = radii
    result_df.index = chain2_arr[:, 0]

    return result_df

In [8]:
def fetch_helices(pdb_name, chain):
    helix_list = []
    
    ### 1. Fetch whole PDB, multiple chains
    coord = PandasPdb()
    coord.fetch_pdb(pdb_name)
    p1 = coord.df['ATOM']
    # p1_CA = p1[p1['atom_name']== 'CA']
    
    ### 2. Fetch single chain
    c_id = coord.df['ATOM']['chain_id'].unique()
    print(c_id)
    c1 = coord.df['ATOM'][coord.df['ATOM']['chain_id']==chain]
    # c1_CA = c1[c1['atom_name']== 'CA']
    
    ### 3. Fetch alpha helix from chain
    h1 = coord.df['OTHERS'][coord.df['OTHERS']['record_name']=='HELIX']
    h1 = h1['entry'].str.split('\s+', expand=True)[[4,5,8,10]]
    h1.columns = ['chain_id', 'start_res', 'end_res', 'size']
    h1['start_res'] = h1['start_res'].astype('int')
    h1['end_res'] = h1['end_res'].astype('int')
    h1['size'] = h1['size'].astype('int')
    all_helices = h1[h1['chain_id']==chain]
    all_helices.reset_index(drop=True, inplace=True)
    
    ### 4. Fetch dfs from all_helices
    for hel in range(len(all_helices)):
        helix_list.append(c1[(c1['residue_number']>=all_helices['start_res'].iloc[hel]) & (c1['residue_number']<=all_helices['end_res'].iloc[hel])])
        
    return helix_list

In [9]:
def fetch_sheets(pdb_name, chain):
    helix_list = []
    
    ### 1. Fetch whole PDB, multiple chains
    coord = PandasPdb()
    coord.fetch_pdb(pdb_name)
    p1 = coord.df['ATOM']
    # p1_CA = p1[p1['atom_name']== 'CA']
    
    ### 2. Fetch single chain
    c_id = coord.df['ATOM']['chain_id'].unique()
    print(c_id)
    c1 = coord.df['ATOM'][coord.df['ATOM']['chain_id']==chain]
    # c1_CA = c1[c1['atom_name']== 'CA']
    
    ### 3. Fetch alpha helix from chain
    h1 = coord.df['OTHERS'][coord.df['OTHERS']['record_name']=='SHEET']
    h1 = h1['entry'].str.split('\s+', expand=True)[[4,5,8,10]]
    h1.columns = ['chain_id', 'start_res', 'end_res', 'size']
    h1['start_res'] = h1['start_res'].astype('int')
    h1['end_res'] = h1['end_res'].astype('int')
    h1['size'] = h1['size'].astype('int')
    all_helices = h1[h1['chain_id']==chain]
    all_helices.reset_index(drop=True, inplace=True)
    
    ### 4. Fetch dfs from all_helices
    for hel in range(len(all_helices)):
        helix_list.append(c1[(c1['residue_number']>=all_helices['start_res'].iloc[hel]) & (c1['residue_number']<=all_helices['end_res'].iloc[hel])])
        
    return helix_list

In [10]:
# pdb_name = "1IGT"
# chain = "A"

In [11]:
# helix_list = []
    
# ### 1. Fetch whole PDB, multiple chains
# coord = PandasPdb()
# coord.fetch_pdb(pdb_name)
# p1 = coord.df['ATOM']
# # p1_CA = p1[p1['atom_name']== 'CA']
    
# ### 2. Fetch single chain
# c_id = coord.df['ATOM']['chain_id'].unique()
# print(c_id)
# c1 = coord.df['ATOM'][coord.df['ATOM']['chain_id']==chain]
# # c1_CA = c1[c1['atom_name']== 'CA']

In [12]:
# ### 3. Fetch alpha helix from chain
# h1 = coord.df['OTHERS'][coord.df['OTHERS']['record_name']=='SHEET']
# h1 = h1['entry'].str.split('\s+', expand=True)[[4,5,8,10]]
# h1.columns = ['chain_id', 'start_res', 'end_res', 'size']
# h1['start_res'] = h1['start_res'].astype('int')
# # h1['end_res'] = h1['end_res'].astype('int')
# # h1['size'] = h1['size'].astype('int')
# # all_helices = h1[h1['chain_id']==chain]
# # all_helices.reset_index(drop=True, inplace=True)
    
# # ### 4. Fetch dfs from all_helices
# # for hel in range(len(all_helices)):
# #     helix_list.append(c1[(c1['residue_number']>=all_helices['start_res'].iloc[hel]) & (c1['residue_number']<=all_helices['end_res'].iloc[hel])])

In [13]:
# Path to your CSV file with escaped backslashes
file_path = "/home/nmoudgal/OPD_Project/DB5.csv"

# Load the CSV file into a NumPy array
data = np.asarray(pd.read_csv(file_path, delimiter=','))

In [ ]:
results_1 = []  # List to store length, hcm, and opd values for the first interacting partner

for i in range(len(data)):
    pdb_name = data[i][0]
    chain1 = data[i][1]
    
    try:
        array = fetch_helices(pdb_name, chain1)
        for j in range(len(array)):
            coords = array[j]
            coords1 = coords[coords["atom_name"] == "CA"]
            coords1 = coords1.loc[:, ["x_coord", "y_coord", "z_coord"]].to_numpy()
            coords = coords.loc[:, ["x_coord", "y_coord", "z_coord"]].to_numpy()
            hcm_value = hcm(coords)
            osipov_value = osipov_whole(coords)
            length = len(coords1)
            results_1.append([length, hcm_value, osipov_value])  
            print("success")
    except:
        print("error")
    if i % 10 == 0 and i > 0:
        results_1_df = pd.DataFrame(results_1, columns=["Length", "HCM", "OPD"])
        results_1_df.to_csv("helix_chir_new.csv", index=False)
        print("Saved!")

['A' 'B' 'C' 'D' 'E']
['E' 'I']
success
success
['A' 'B' 'C' 'D' 'E' 'F']
['A' 'B' 'C' 'D']
success
success
success
['A' 'B' 'C' 'D' 'E']
['A' 'D']
error
['A' 'B']
success
success
success
success
['A' 'B']
success
success
['A' 'B' 'C']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']
success
success
success
['R' 'S']
success
success
success
success
success
success
Saved!
['A' 'B']
success
success


In [ ]:
results_df = pd.DataFrame(results, columns=["HCM", "Osipov Whole"])        
results_df.to_csv("results.csv", index=False)

In [ ]:
x = results[:, 0]  # First column
y = results[:, 1]  # Second column

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, s=10, alpha=0.8)  # Adjust `s` and `alpha` for point size and transparency
plt.title("HCM vs. OPD for Naturally Occurring Alpha Helices", fontsize=16)
plt.xlabel("HCM", fontsize=14)
plt.ylabel("OPD", fontsize=14)
m, b = np.polyfit(x, y, 1)  # Linear fit (degree 1)

# Plot the line of best fit
plt.plot(x, m * x + b, color="red", label="Best Fit Line", linewidth=2)


plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
results = np.array(results)

In [ ]:
x = results[:, 0]  # First column
y = np.abs(results[:, 1])  # Second column

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, s=10, alpha=0.8)  # Adjust `s` and `alpha` for point size and transparency
plt.title("HCM vs. OPD for Naturally Occurring Alpha Helices", fontsize=16)
plt.xlabel("HCM", fontsize=14)
plt.ylabel("Absolute OPD", fontsize=14)
m, b = np.polyfit(x, y, 1)  # Linear fit (degree 1)

# Plot the line of best fit
plt.plot(x, m * x + b, color="red", label="Best Fit Line", linewidth=2)


plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
np.corrcoef(x, y)[0, 1]